# STT sample

## Ses Kaydını Metne Çeviren Uygulama Örneği

Model: [openai/whisper-base](https://huggingface.co/openai/whisper-base)

Dataset: [Nexdata/English_Emotional_Speech_Data_by_Microphone](https://huggingface.co/datasets/Nexdata/English_Emotional_Speech_Data_by_Microphone/viewer/default/train?views%5B%5D=train&row=0)

In [1]:
import torch
print("CUDA durumu:", torch.cuda.is_available())
print("Kullanılan cihaz:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


CUDA durumu: True
Kullanılan cihaz: cuda


In [2]:
!pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 41.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=128a2857166c885eed25c4aa1666192197a37ff414ef29b1446dacae0e681593
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [4]:
import whisper

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

result = model.transcribe("/content/audio.wav")
print(result["text"])


 After the accident, Bill's sister is finally feeling better.
